In [213]:
import json

file_path = "/Users/YannanGao/NLP_Project2/project-files/train.json"


file = open(file_path)
content = json.load(file)
print(len(content))
# print(content["train-0"])

1168


In [214]:
## All the text information stored in context_list
context_list = []
label_list = []
for i in range(len(content)):
    title = "train-{}".format(i)
    context_list.append(content[title]["text"])
    label_list.append(content[title]["label"])
print(len(context_list))
print(len(label_list))
print(context_list[0])

1168
1168
why houston flooding isn‘t a sign of climate change
Distinguished US climate scientist, Dr Roy Spencer writes: "In the context of climate change, is what we are seeing in Houston a new level of disaster which is becoming more common? The flood disaster unfolding in Houston is certainly very unusual. But so are other natural weather disasters, which have always occurred and always will occur....Major floods are difficult to compare throughout history because of the ways we alter the landscape., For example, as cities like Houston expand over the years, soil is covered up by roads, parking lots and buidings, with water rapidly draining off rather than soaking into the soil. The population of Houston is now ten times what is was in the 1920s.  The Houston metroplex has expanded greatly and the water drainage is basically in the direction of downtown Houston."


In [215]:
# Combine the title with the content
import re

def reformat_content(contents):
    concat_contents = []
    for content in contents:
        concat_content = re.sub("\n", ". ", content)
        concat_contents.append(concat_content)
    return concat_contents

re_contents = reformat_content(context_list)
print(len(re_contents))
print(re_contents[0])

1168
why houston flooding isn‘t a sign of climate change. Distinguished US climate scientist, Dr Roy Spencer writes: "In the context of climate change, is what we are seeing in Houston a new level of disaster which is becoming more common? The flood disaster unfolding in Houston is certainly very unusual. But so are other natural weather disasters, which have always occurred and always will occur....Major floods are difficult to compare throughout history because of the ways we alter the landscape., For example, as cities like Houston expand over the years, soil is covered up by roads, parking lots and buidings, with water rapidly draining off rather than soaking into the soil. The population of Houston is now ten times what is was in the 1920s.  The Houston metroplex has expanded greatly and the water drainage is basically in the direction of downtown Houston."


In [222]:
# Expand the training data set with external data
import pandas as pd
import nltk

negative_data = pd.read_csv("text.csv")
print(negative_data.shape)

negative_content = []
for i in range(len(negative_data)):
    text = negative_data.iloc[i, 1]
    tokens = nltk.word_tokenize(text)
    if len(tokens) >= 50:
        negative_content.append(text)
negative_content.extend(negative_content * 2)
print(len(negative_content))

(407, 2)
546


In [223]:
print(negative_content[370])

Syria signs Paris Climate Agreement leaving the US as the only country in the world not in it. Syria has become a signatory of the Paris climate agreement, leaving the US as the only country in the world not to support the framework deal to combat greenhouse gas emissions. When President Donald Trump announced he intended to pull the US out of the agreement, it initially meant America would join Nicaragua and Syria on a small list of countries who were not part of the deal.


In [224]:
def normalize(corpus):
    normalized_corpus = []
    punctuation = "*"
    for text in corpus:
        text = text.lower()
        text = re.sub(r"<br />", r" ", text)
        text = re.sub(' +', ' ', text)
        text = re.sub(r'(\W)(?=\1)', ' ', text)
        text = re.sub(r"[|\t]", ',', text) 
        text = re.sub(r'\s\s+', ' ', text).lstrip(' ')
        text = re.sub(r'[{}]+'.format(punctuation),' ',text)
        
        normalized_corpus.append(text)
    return normalized_corpus

external_neg_norm_data = normalize(negative_content)
print(len(external_neg_norm_data))
re_contents_norm = normalize(re_contents)
print(len(re_contents_norm))

546
1168


In [225]:
print(re_contents_norm[123])

the heartland institute recently released a 52-page report by jay lehr, ph.d., the organization’s science director, and 18 distinguished climate and weather researchers, providing an extensive, detailed critique of the november 2017 “climate science special report” (cssr) produced by the u.s. global change research program (usgcrp) . the taskforce assembled by lehr shows cssr suffers from the same “shortcomings and biases apparent in the previous work produced by the usgcrp.” among the fallacies cssr perpetuates is the misperception “the science is settled” that human activities are driving climate change and climate change is necessarily catastrophic for humanity and necessitates a wholesale takeover by governments to reshape the economy sans the use of fossil fuels . among the “foundational problems” with cssr is the fact its authors hide the myriad scientific uncertainties about the causes and consequences of ongoing climate change. multiple lines of evidence show the climate is muc

In [226]:
print(external_neg_norm_data[123])

the main contributor to warming over the last 170 years is human influence on climate from increasing greenhouse gases in the atmosphere,” the met office said on wednesday. global temperatures in 2019 were on average 1.05c above pre-industrial levels and this year could be hotter, the british weather service said.


In [227]:
# Randomly select 1168 neg samples
import random

select_neg_data = random.sample(external_neg_norm_data, 500)
select_pos_data = random.sample(re_contents_norm, 500)
print(len(select_neg_data))
print(len(select_pos_data))

500
500


In [228]:
# Concat the positive data with the external negative data
train_contents = select_pos_data + select_neg_data
print(len(train_contents))

1000


In [229]:
# Concat the negative labels into the label_list
train_label_list = []
train_label_list.extend([1] * len(select_pos_data))
train_label_list.extend([0] * len(select_neg_data))
print(len(train_label_list))

1000


In [ ]:
import nltk

content_length = []
for content in re_contents:
    tokens = nltk.word_tokenize(content)
    content_length.append(len(tokens))
print(content_length)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 10))
plt.hist(content_length, bins=100)
plt.show()

In [231]:
# Now we gonna load the module
!pip install transformers

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3 -m pip install --upgrade pip' command.


In [232]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [233]:
import nltk
import ssl


ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
sent_segmenter = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /Users/troycao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [234]:
# Here we gonna get the ids of all the contents

def convert_para_to_id(contents, para_length):
    contents_ids_list = []
    attentions_list = []
    for content in contents:
        word_count = 0
        content_list = []
        attention_list = []
        sentences = sent_segmenter.tokenize(content)
        for sentence in sentences:
            while word_count < para_length:
                sentence = re.sub(r"[^a-zA-Z0-9]", "", sentence)
                encoded_con = tokenizer.encode(sentence, add_special_tokens=True)
                content_list.extend(encoded_con)
                word_count += len(encoded_con)
        if len(content_list) > para_length:
            content_list = content_list[:para_length]
            attention_list.extend([1] * para_length)
        else:
            content_list.extend([0] * (para_length - len(content_list)))
            attention_list.extend([1] * len(content_list))
            attention_list.extend([0] * (para_length - len(content_list)))
        contents_ids_list.append(content_list)
        attentions_list.append(attention_list)
    return contents_ids_list, attentions_list

ids_list, attention_list = convert_para_to_id(train_contents, 50)
print(len(ids_list))
print(len(attention_list))

1000
1000


In [235]:
# Covert label list to Tensor
import torch

labels = torch.LongTensor(train_label_list)
input_ids = torch.LongTensor(ids_list)
attention_mask = torch.LongTensor(attention_list)

In [236]:
# Combine the traiing inputs into a TensroDataset
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

train_data = TensorDataset(input_ids, attention_mask, labels)


train_data_loader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=10)
print(len(train_data_loader))

100


In [237]:
# Load the Model
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels = 2,
        output_attentions = False,
        output_hidden_states = False
)

In [238]:
#Define the optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5, eps=1e-8)


In [239]:
from transformers import get_linear_schedule_with_warmup

epochs = 2

total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [240]:
# Define a helper function for calculating Accuracy
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [207]:
# get the dev data
def read_dev_file(file_path):
    file = open(file_path)
    contents_list = []
    labels_list = []
    content = json.load(file)
    for i in range(len(content)):
        title = "dev-{}".format(i)
        contents_list.append(content[title]["text"])
        labels_list.append(content[title]["label"])
    return contents_list, labels_list

dev_file_path = "/Users/troycao/Documents/GitHub/NLP_Project2/project-files/dev.json"
dev_contents_list, dev_labels_list = read_dev_file(dev_file_path)

dev_re_contents = reformat_content(dev_contents_list)
dev_re_contents_norm = normalize(dev_re_contents)
print(len(dev_re_contents_norm))

100


In [241]:
dev_ids_list, dev_attention_list = convert_para_to_id(dev_re_contents_norm, 50)

dev_labels = torch.LongTensor(dev_labels_list)

dev_input_ids = torch.LongTensor(dev_ids_list)
dev_attention_mask = torch.LongTensor(dev_attention_list)

In [242]:
dev_data = TensorDataset(dev_input_ids, dev_attention_mask, dev_labels)


dev_data_loader = DataLoader(dev_data, sampler=RandomSampler(dev_data), batch_size=10)
print(len(dev_data_loader))

10


In [243]:
# Now training
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

training_stats = [] # used to store the training information


for epoch_i in range(epochs):
    
    print("")
    print("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
    print("Training...")
    
    total_train_loss = 0
    
    model.train()
    
    for step, batch in enumerate(train_data_loader):
        
        if step % 10 == 0 and not step == 0:
            print("Batch {} of {}".format(step, len(train_data_loader)))
        
        batch_input_ids = torch.LongTensor(batch[0])
        batch_input_mask = torch.LongTensor(batch[1])
        batch_labels = batch[2]
        
        model.zero_grad()
        loss, logits = model(batch_input_ids, token_type_ids=None,
                            attention_mask=batch_input_mask,
                            labels=batch_labels)
        total_train_loss += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
        
        optimizer.step()
        
        scheduler.step()
    
    avg_train_loss = total_train_loss / len(train_data_loader)
    
    print("")
    print(" Average Training Loss is {:2f}".format(avg_train_loss))
    
    # Now perform validation
    
    print("")
    print("Running Validation...")
    
    model.eval()
    
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    for batch in dev_data_loader:
        dev_b_input_ids = batch[0]
        dev_b_input_mask = batch[1]
        dev_b_labels = batch[2]
        
        with torch.no_grad():
            loss, logits = model(dev_b_input_ids, 
                                 token_type_ids=None, 
                                attention_mask=dev_b_input_mask,
                                labels=dev_b_labels)
        total_eval_loss += loss.item()
        logits = logits.detach().numpy()
        label_ids = dev_b_labels.numpy()
        
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_loss = total_eval_loss / len(dev_data_loader)
    avg_val_accuracy = total_eval_accuracy / len(dev_data_loader)
    print("Validation loss :{}".format(avg_val_loss))
    print("Accuracy is {}".format(avg_val_accuracy))
    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy
        }
    )
    
print("")
print("Training Complete!...")


======== Epoch 1 / 2 ========
Training...


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Batch 10 of 100
Batch 20 of 100
Batch 30 of 100
Batch 40 of 100
Batch 50 of 100
Batch 60 of 100
Batch 70 of 100
Batch 80 of 100
Batch 90 of 100

 Average Training Loss is 0.609810

Running Validation...
Validation loss :0.9584946751594543
Accuracy is 0.48999999999999994

======== Epoch 2 / 2 ========
Training...
Batch 10 of 100
Batch 20 of 100
Batch 30 of 100
Batch 40 of 100
Batch 50 of 100
Batch 60 of 100
Batch 70 of 100
Batch 80 of 100
Batch 90 of 100

 Average Training Loss is 0.488814

Running Validation...
Validation loss :1.324871677160263
Accuracy is 0.4800000000000001

Training Complete!...


In [ ]:
torch.save(model, "BERT_v6.0.1pkl")

In [ ]:
import pandas as pd

pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

df_stats.to_csv("Bert_version5.0.csv", index=True, sep=",")

In [211]:
from sklearn.metrics import precision_score, recall_score, f1_score

bert_test_model = torch.load("./meta_data/BERT_v6.0.pkl")

bert_test_model.eval()

with torch.no_grad():
    test_loss, test_logits = bert_test_model(dev_input_ids, token_type_ids=None,
                            attention_mask=dev_attention_mask,
                            labels=dev_labels)
test_pred = np.argmax(test_logits, axis=1).flatten()
actual_pred = dev_labels.flatten()
print(test_pred)

p = precision_score(test_pred, actual_pred)
print("The Precision is: {:.4f}".format(p))

r = recall_score(test_pred, actual_pred)
print("The Recall is: {:.4f}".format(r))

f1_score = f1_score(test_pred,actual_pred, average='macro')
print("The Macro_F1_Score is: {:.4f}".format(f1_score))

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])
The Precision is: 0.9400
The Recall is: 0.5663
The Macro_F1_Score is: 0.5623


In [212]:
print(actual_pred)

tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 1])


In [ ]:
#Load test data
def read_test_file(file_path):
    file = open(file_path)
    contents_list = []
    labels_list = []
    content = json.load(file)
    for i in range(len(content)):
        title = "test-{}".format(i)
        contents_list.append(content[title]["text"])
    return contents_list

test_content = read_test_file("/Users/troycao/Documents/GitHub/NLP_Project2/project-files/test-unlabelled.json")
test_re_contents = reformat_content(test_content)
test_re_contents_norm = normalize(test_re_contents)

print(len(test_re_contents_norm))

In [ ]:
test_ids_list, test_attention_list = convert_para_to_id(test_re_contents_norm, 80)

test_input_ids = torch.LongTensor(test_ids_list)
test_attention_mask = torch.LongTensor(test_attention_list)
temp_labels = []
temp_labels.extend([1] * len(test_re_contents_norm))
test_labels = torch.LongTensor(temp_labels)

In [ ]:
print(test_labels.shape)

In [ ]:
test_model = torch.load("./meta_data/BERT_v6.0.1pkl")
test_model.eval()
with torch.no_grad():
    test_loss, test_logits = test_model(test_input_ids, token_type_ids=None,
                            attention_mask=test_attention_mask,
                            labels=test_labels)

test_pred = np.argmax(test_logits, axis=1).flatten()
test_result = {}

for i in range(len(test_pred)):
    title = "test-{}".format(i)
    test_result[title] = {}
    test_result[title]["label"] = test_pred[i]
print(test_result["test-0"])


In [ ]:
for i in range(len(test_result)):
    title = "test-{}".format(i)
    item = test_result[title]["label"]
    test_result[title]["label"] = int(item.numpy())
print(test_result["test-0"])

In [ ]:
with open("test-output.json", "w") as f:
     json.dump(test_result, f)